# Import necessary packages

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics import mean_absolute_error, mean_squared_error

from timeit import default_timer as timer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Import original data and preprocessed data

In [ ]:
sales_train_val = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/sales_train_validation.csv')

### Note: This Step is for the extraction of California State and Foods category 

In [ ]:
sales_train_val = sales_train_val[(sales_train_val['state_id'] == 'CA') & (sales_train_val['cat_id'] == 'FOODS')]

In [ ]:
INPUT_DIR_2 = '/content/drive/MyDrive/Colab Notebooks/List of Product ID according to 4 demand patterns/California/Foods/2-year'
list_intermittent = pd.read_csv(f'{INPUT_DIR_2}/Intermittent_ID_2_Year_Data.csv')
list_lumpy = pd.read_csv(f'{INPUT_DIR_2}/Lumpy_ID_2_Year_Data.csv')
list_erratic = pd.read_csv(f'{INPUT_DIR_2}/Erratic_ID_2_Year_Data.csv')
list_smooth = pd.read_csv(f'{INPUT_DIR_2}/Smooth_ID_2_Year_Data.csv')

list_intermittent = list_intermittent['0'].values.tolist()
list_lumpy = list_lumpy['0'].values.tolist()
list_erratic = list_erratic['0'].values.tolist()
list_smooth = list_smooth['0'].values.tolist()

sales_intermittent = sales_train_val[sales_train_val.id.isin(list_intermittent)]
sales_lumpy = sales_train_val[sales_train_val.id.isin(list_lumpy)]
sales_erratic = sales_train_val[sales_train_val.id.isin(list_erratic)]
sales_smooth = sales_train_val[sales_train_val.id.isin(list_smooth)]

# User-defined functions to calculate Metrics and Croston_TSB algorithm

In [ ]:
ROUNDING_DECIMAL = 4

def mase_calculation(ts, prediction):
    divisor = 0
    for i in range(1, ts.shape[0]):
        divisor = divisor + abs(ts.iloc[i] - ts.iloc[i-1])
    divisor = divisor/(ts.shape[0] - 1)
    diff    = abs(ts - prediction[:ts.shape[0]])/divisor
    mase    = diff.mean()
    return mase

def mape_calculation(actual, pred): 
    if not all([isinstance(actual, np.ndarray), isinstance(pred, np.ndarray)]):
        actual, pred = np.array(actual), np.array(pred)
    mask = (actual != 0)
    return round((np.fabs(actual - pred)/actual)[mask].mean()*100, ROUNDING_DECIMAL)

def wmape_calculation(actual, pred):
    if not all([isinstance(actual, np.ndarray), isinstance(pred, np.ndarray)]):
        actual, pred = np.array(actual), np.array(pred)
    return round((np.sum(np.absolute(actual-pred))/np.sum(actual))*100, ROUNDING_DECIMAL)

def smape_calculation(actual, predicted):
    if not all([isinstance(actual, np.ndarray), isinstance(predicted, np.ndarray)]):
        actual, predicted = np.array(actual), np.array(predicted)
    return round(np.mean(np.abs(predicted - actual) / ((np.abs(predicted) + np.abs(actual))/2))*100, ROUNDING_DECIMAL)

In [ ]:
def Croston_TSB(ts,extra_periods=1,alpha=0.4,beta=0.4):
    d = np.array(ts) # Transform the input into a numpy array
    cols = len(d) # Historical period length
    d = np.append(d,[np.nan]*extra_periods) # Append np.nan into the demand array to cover future periods
    
    #level (a), probability(p) and forecast (f)
    a,p,f = np.full((3,cols+extra_periods),np.nan)
# Initialization
    first_occurence = np.argmax(d[:cols]>0)
    a[0] = d[first_occurence]
    p[0] = 1/(1 + first_occurence)
    f[0] = p[0]*a[0]
                 
    # Create all the t+1 forecasts
    for t in range(0,cols): 
        if d[t] > 0:
            a[t+1] = alpha*d[t] + (1-alpha)*a[t] 
            p[t+1] = beta*(1) + (1-beta)*p[t]  
        else:
            a[t+1] = a[t]
            p[t+1] = (1-beta)*p[t]       
        f[t+1] = p[t+1]*a[t+1]
        
    # Future Forecast
    a[cols+1:cols+extra_periods] = a[cols]
    p[cols+1:cols+extra_periods] = p[cols]
    f[cols+1:cols+extra_periods] = f[cols]
                      
    df = pd.DataFrame.from_dict({"Demand":d,"Forecast":f,"Period":p,"Level":a,"Error":d-f})
    return df

# Set value for parameters

In [ ]:
list_params_alpha = [round(item, 1) for item in list(np.arange(0.1, 1, 0.1))]
list_params_beta = [round(item, 1) for item in list(np.arange(0.1, 1, 0.1))]

In [ ]:
start_train_date = '2014-04-11'
end_train_date = '2016-04-10'
n_pred_days = 14
num_train_needed = 100
validation_training_ratio = 0.95

# Pipeline for Croston_TSB

In [ ]:
def CrostonTSB_output_all_params(sales_pattern, list_params_alpha, list_params_beta, 
                                 start_train_date, end_train_date, n_pred_days, 
                                 num_train_needed, validation_training_ratio):
    
    sales_pattern_py = sales_pattern.copy()
    list_pattern_py = sales_pattern_py.id.unique().tolist()
    
    sales_pattern_py = sales_pattern_py.drop(['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], axis=1)
    df_pattern = sales_pattern_py.melt(['id'], var_name='Date').pivot(index = ['Date'], columns = 'id', values = 'value').reset_index()
    df_pattern.drop('Date', axis=1, inplace=True)
    df_pattern.index = pd.date_range('2011-01-29', periods=df_pattern.shape[0], freq="D")
    df_pattern.index.names = ['Date']
    df_pattern = df_pattern.astype('float64')
    
    train_data = df_pattern[(df_pattern.index >= start_train_date) & (df_pattern.index <= end_train_date)]
    test_data = df_pattern.iloc[n_pred_days*-1:]
    train_data_first_part = train_data.iloc[:int(len(train_data)*validation_training_ratio), :]
    train_data_second_part = train_data.iloc[int(len(train_data)*validation_training_ratio):, :]
    
    df_result_all_params = pd.DataFrame()
    for product in list_pattern_py:
        train_first = train_data_first_part[product]
        train_second = train_data_second_part[product]
        for i in list_params_alpha:
            for j in list_params_beta:
                predictions = list()
                history = [x for x in train_first[num_train_needed*-1:]]
                for t in range(len(train_second)):
                    yhat = Croston_TSB(history, extra_periods=1, alpha = i, beta = j)['Forecast'].iloc[-1]
                    predictions.append(yhat)
                    history.append(train_second[t])
                df_result_temp = pd.DataFrame({'Product': [product for count in range(len(train_second))],
                                               'Actual Data': train_second,
                                               'Forecast': predictions,
                                               'Alpha': [i for count_i in range(len(train_second))],
                                               'Beta': [j for count_j in range(len(train_second))]})
                df_result_all_params = df_result_all_params.append(df_result_temp, ignore_index=True)
            
    return df_result_all_params

In [ ]:
start = timer()

pattern_df_result_all_params = CrostonTSB_output_all_params(sales_smooth,
                                                          list_params_alpha,
                                                          list_params_beta,
                                                          start_train_date,
                                                          end_train_date, 
                                                          n_pred_days,
                                                          num_train_needed,
                                                          validation_training_ratio)

end = timer()

print('This line of code took {} minutes'.format((end-start) / 60))

This line of code took 40.409327928083336 minutes


In [ ]:
pattern_df_result_all_params

,Product,Actual Data,Forecast,Alpha,Beta
0,FOODS_1_004_CA_1_validation,7.0,8.004796,0.1,0.1
1,FOODS_1_004_CA_1_validation,11.0,8.011649,0.1,0.1
2,FOODS_1_004_CA_1_validation,7.0,8.305201,0.1,0.1
3,FOODS_1_004_CA_1_validation,7.0,8.253402,0.1,0.1
4,FOODS_1_004_CA_1_validation,11.0,8.191825,0.1,0.1
...,...,...,...,...,...
1924069,FOODS_3_818_CA_4_validation,5.0,1.883278,0.9,0.9
1924070,FOODS_3_818_CA_4_validation,2.0,4.685817,0.9,0.9
1924071,FOODS_3_818_CA_4_validation,0.0,2.268800,0.9,0.9
1924072,FOODS_3_818_CA_4_validation,3.0,0.226880,0.9,0.9


In [ ]:
pattern_df_result_all_params.to_csv('CrostonTSB_Smooth_All_Parameters_Hyperparameter_Tuning_Data.csv')

# Summary all metrics

In [ ]:
def get_metrics_result_all_params(data):
    metrics_df = {}
    metrics_df['MASE'] = mase_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['WMAPE'] = wmape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['SMAPE'] = smape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['MAPE'] = mape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['MAE'] = mean_absolute_error(data['Actual Data'], data['Forecast'])
    metrics_df['RMSE'] = np.sqrt(mean_squared_error(data['Actual Data'], data['Forecast']))
    return pd.Series(metrics_df)

In [ ]:
start = timer()

df_result_metrics_all_params = pattern_df_result_all_params.groupby(['Product', 'Alpha', 'Beta']).apply(get_metrics_result_all_params).reset_index()

end = timer()
print('This line of code took {} minutes'.format((end-start) / 60))

This line of code took 2.1866917593999915 minutes


In [ ]:
df_result_metrics_all_params

,Product,Alpha,Beta,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
0,FOODS_1_004_CA_1_validation,0.1,0.1,0.723248,55.7503,62.4456,103.5057,3.073802,3.819768
1,FOODS_1_004_CA_1_validation,0.1,0.2,0.774853,59.7282,64.3611,111.6832,3.293124,4.050079
2,FOODS_1_004_CA_1_validation,0.1,0.3,0.793995,61.2038,65.1917,113.9040,3.374478,4.136095
3,FOODS_1_004_CA_1_validation,0.1,0.4,0.808302,62.3066,65.8981,115.3864,3.435285,4.196297
4,FOODS_1_004_CA_1_validation,0.1,0.5,0.819639,63.1805,66.4837,116.4984,3.483467,4.251259
...,...,...,...,...,...,...,...,...,...
51997,FOODS_3_823_CA_3_validation,0.9,0.5,inf,inf,200.0000,NaN,0.029984,0.105302
51998,FOODS_3_823_CA_3_validation,0.9,0.6,inf,inf,200.0000,NaN,0.015345,0.061106
51999,FOODS_3_823_CA_3_validation,0.9,0.7,inf,inf,200.0000,NaN,0.006473,0.028895
52000,FOODS_3_823_CA_3_validation,0.9,0.8,inf,inf,200.0000,NaN,0.001918,0.009526


# Check what products has unexpected metrics result

In [ ]:
df_result_metrics_all_params[df_result_metrics_all_params['MASE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,Beta,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
4779,FOODS_2_024_CA_2_validation,0.1,0.1,NaN,NaN,NaN,NaN,0.000000,0.000000
4780,FOODS_2_024_CA_2_validation,0.1,0.2,NaN,NaN,NaN,NaN,0.000000,0.000000
4781,FOODS_2_024_CA_2_validation,0.1,0.3,NaN,NaN,NaN,NaN,0.000000,0.000000
4782,FOODS_2_024_CA_2_validation,0.1,0.4,NaN,NaN,NaN,NaN,0.000000,0.000000
4783,FOODS_2_024_CA_2_validation,0.1,0.5,NaN,NaN,NaN,NaN,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...
51997,FOODS_3_823_CA_3_validation,0.9,0.5,inf,inf,200.0,NaN,0.029984,0.105302
51998,FOODS_3_823_CA_3_validation,0.9,0.6,inf,inf,200.0,NaN,0.015345,0.061106
51999,FOODS_3_823_CA_3_validation,0.9,0.7,inf,inf,200.0,NaN,0.006473,0.028895
52000,FOODS_3_823_CA_3_validation,0.9,0.8,inf,inf,200.0,NaN,0.001918,0.009526


In [ ]:
df_result_metrics_all_params[df_result_metrics_all_params['WMAPE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,Beta,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
4779,FOODS_2_024_CA_2_validation,0.1,0.1,NaN,NaN,NaN,NaN,0.000000,0.000000
4780,FOODS_2_024_CA_2_validation,0.1,0.2,NaN,NaN,NaN,NaN,0.000000,0.000000
4781,FOODS_2_024_CA_2_validation,0.1,0.3,NaN,NaN,NaN,NaN,0.000000,0.000000
4782,FOODS_2_024_CA_2_validation,0.1,0.4,NaN,NaN,NaN,NaN,0.000000,0.000000
4783,FOODS_2_024_CA_2_validation,0.1,0.5,NaN,NaN,NaN,NaN,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...
51997,FOODS_3_823_CA_3_validation,0.9,0.5,inf,inf,200.0,NaN,0.029984,0.105302
51998,FOODS_3_823_CA_3_validation,0.9,0.6,inf,inf,200.0,NaN,0.015345,0.061106
51999,FOODS_3_823_CA_3_validation,0.9,0.7,inf,inf,200.0,NaN,0.006473,0.028895
52000,FOODS_3_823_CA_3_validation,0.9,0.8,inf,inf,200.0,NaN,0.001918,0.009526


In [ ]:
df_result_metrics_all_params[df_result_metrics_all_params['SMAPE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,Beta,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
4779,FOODS_2_024_CA_2_validation,0.1,0.1,NaN,NaN,NaN,NaN,0.000000,0.000000
4780,FOODS_2_024_CA_2_validation,0.1,0.2,NaN,NaN,NaN,NaN,0.000000,0.000000
4781,FOODS_2_024_CA_2_validation,0.1,0.3,NaN,NaN,NaN,NaN,0.000000,0.000000
4782,FOODS_2_024_CA_2_validation,0.1,0.4,NaN,NaN,NaN,NaN,0.000000,0.000000
4783,FOODS_2_024_CA_2_validation,0.1,0.5,NaN,NaN,NaN,NaN,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...
50782,FOODS_3_795_CA_4_validation,0.9,0.5,0.808926,96.9962,NaN,72.1230,0.629164,1.058291
50783,FOODS_3_795_CA_4_validation,0.9,0.6,0.829640,99.4800,NaN,74.7928,0.645276,1.077440
50784,FOODS_3_795_CA_4_validation,0.9,0.7,0.853574,102.3498,NaN,77.9784,0.663891,1.100488
50785,FOODS_3_795_CA_4_validation,0.9,0.8,0.877730,105.2463,NaN,80.9685,0.682679,1.128893


In [ ]:
df_result_metrics_all_params[df_result_metrics_all_params['MAPE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,Beta,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
4779,FOODS_2_024_CA_2_validation,0.1,0.1,NaN,NaN,NaN,NaN,0.000000,0.000000
4780,FOODS_2_024_CA_2_validation,0.1,0.2,NaN,NaN,NaN,NaN,0.000000,0.000000
4781,FOODS_2_024_CA_2_validation,0.1,0.3,NaN,NaN,NaN,NaN,0.000000,0.000000
4782,FOODS_2_024_CA_2_validation,0.1,0.4,NaN,NaN,NaN,NaN,0.000000,0.000000
4783,FOODS_2_024_CA_2_validation,0.1,0.5,NaN,NaN,NaN,NaN,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...
51997,FOODS_3_823_CA_3_validation,0.9,0.5,inf,inf,200.0,NaN,0.029984,0.105302
51998,FOODS_3_823_CA_3_validation,0.9,0.6,inf,inf,200.0,NaN,0.015345,0.061106
51999,FOODS_3_823_CA_3_validation,0.9,0.7,inf,inf,200.0,NaN,0.006473,0.028895
52000,FOODS_3_823_CA_3_validation,0.9,0.8,inf,inf,200.0,NaN,0.001918,0.009526


In [ ]:
df_result_metrics_all_params[df_result_metrics_all_params['RMSE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,Beta,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE


In [ ]:
df_result_metrics_all_params[df_result_metrics_all_params['MAE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,Beta,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE


# Metrics Statistics

In [ ]:
list_metrics = ['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE']

In [ ]:
for mts in list_metrics:
    print('Percentage of unexpected values of', mts, 'is: {}'.format(df_result_metrics_all_params[df_result_metrics_all_params[mts].isin([np.nan, np.inf, -np.inf])].Product.nunique() / df_result_metrics_all_params.Product.nunique() * 100), "%")

Percentage of unexpected values of MASE is: 10.903426791277258 %
Percentage of unexpected values of WMAPE is: 10.903426791277258 %
Percentage of unexpected values of SMAPE is: 11.838006230529595 %
Percentage of unexpected values of MAPE is: 10.903426791277258 %
Percentage of unexpected values of MAE is: 0.0 %
Percentage of unexpected values of RMSE is: 0.0 %


### Filter all rows that have unexpected metrics values

In [ ]:
df_result_metrics_all_params = df_result_metrics_all_params[~df_result_metrics_all_params.isin([np.nan, np.inf, -np.inf]).any(1)]

### Get MEAN metrics value of each Alpha

In [ ]:
df_result_metrics_all_params.groupby(['Alpha', 'Beta'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].mean()

MASE      WMAPE      SMAPE       MAPE       MAE      RMSE
Alpha Beta                                                               
0.1   0.1   0.835236  67.680626  77.194002  64.318207  3.762564  4.838125
      0.2   0.829499  67.095952  77.477289  65.194899  3.768644  4.856986
      0.3   0.829429  67.096714  77.993956  66.039068  3.790018  4.888811
      0.4   0.831397  67.246540  78.680073  67.014751  3.821563  4.926657
      0.5   0.834582  67.492970  79.519376  68.082155  3.855872  4.969434
...              ...        ...        ...        ...       ...       ...
0.9   0.5   0.938712  75.080088  86.907003  77.172061  4.162406  5.521556
      0.6   0.944080  75.520070  87.822931  78.330076  4.195515  5.575722
      0.7   0.950523  76.070127  88.993789  79.619222  4.231034  5.636447
      0.8   0.958355  76.763666  90.528806  81.136321  4.268976  5.705356
      0.9   0.967894  77.614634  92.661011  82.985733  4.314963  5.784631

[81 rows x 6 columns]

In [ ]:
for mts in list_metrics:
    print('The optimum Alpha and Beta based on Mean', mts, 'is: {}'.format(df_result_metrics_all_params.groupby(['Alpha', 'Beta'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].mean()[mts].idxmin()))

The optimum Alpha and Beta based on Mean MASE is: (0.1, 0.3)
The optimum Alpha and Beta based on Mean WMAPE is: (0.1, 0.2)
The optimum Alpha and Beta based on Mean SMAPE is: (0.1, 0.1)
The optimum Alpha and Beta based on Mean MAPE is: (0.1, 0.1)
The optimum Alpha and Beta based on Mean MAE is: (0.2, 0.1)
The optimum Alpha and Beta based on Mean RMSE is: (0.1, 0.1)


### Get MEDIAN metrics value of each Alpha

In [ ]:
df_result_metrics_all_params.groupby(['Alpha', 'Beta'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].median()

MASE    WMAPE    SMAPE     MAPE       MAE      RMSE
Alpha Beta                                                         
0.1   0.1   0.792228  58.7351  66.4433  58.7109  2.356284  2.992005
      0.2   0.793499  58.7205  66.7848  59.9225  2.375086  2.984908
      0.3   0.794683  59.0949  67.4145  60.8900  2.365289  2.988199
      0.4   0.799413  60.0326  68.5403  61.8978  2.377044  3.003719
      0.5   0.806338  60.3510  69.4254  63.5497  2.377044  3.037282
...              ...      ...      ...      ...       ...       ...
0.9   0.5   0.937844  69.4340  78.6011  72.4450  2.791492  3.610392
      0.6   0.941287  69.9880  80.0452  73.8029  2.810533  3.663555
      0.7   0.946547  70.3798  81.4571  75.3386  2.823644  3.685434
      0.8   0.953600  70.4071  83.4317  77.1103  2.839483  3.722122
      0.9   0.964339  71.3339  86.1522  79.0477  2.850994  3.770556

[81 rows x 6 columns]

In [ ]:
for mts in list_metrics:
    print('The optimum Alpha and Beta based on Median of', mts, 'is: {}'.format(df_result_metrics_all_params.groupby(['Alpha', 'Beta'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].median()[mts].idxmin()))

The optimum Alpha and Beta based on Median of MASE is: (0.1, 0.1)
The optimum Alpha and Beta based on Median of WMAPE is: (0.1, 0.2)
The optimum Alpha and Beta based on Median of SMAPE is: (0.1, 0.1)
The optimum Alpha and Beta based on Median of MAPE is: (0.1, 0.1)
The optimum Alpha and Beta based on Median of MAE is: (0.1, 0.1)
The optimum Alpha and Beta based on Median of RMSE is: (0.1, 0.2)


# Run Croston model after decide best paramaters

In [ ]:
best_alpha = 0.1
best_beta = 0.1

In [ ]:
def CrostonTSB_test_data(best_alpha, best_beta, sales_pattern,
                         start_train_date, end_train_date, 
                         n_pred_days, num_train_needed, validation_training_ratio):
    
    sales_pattern_py = sales_pattern.copy()
    list_pattern_py = sales_pattern_py.id.unique().tolist()
    
    sales_pattern_py = sales_pattern_py.drop(['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], axis=1)
    df_pattern = sales_pattern_py.melt(['id'], var_name='Date').pivot(index = ['Date'], columns = 'id', values = 'value').reset_index()
    df_pattern.drop('Date', axis=1, inplace=True)
    df_pattern.index = pd.date_range('2011-01-29', periods=df_pattern.shape[0], freq="D")
    df_pattern.index.names = ['Date']
    df_pattern = df_pattern.astype('float64')
    
    train_data = df_pattern[(df_pattern.index >= start_train_date) & (df_pattern.index <= end_train_date)]
    test_data = df_pattern.iloc[n_pred_days*-1:]
    
    df_result_best_params = pd.DataFrame()
    for product in list_pattern_py:
        train_product = train_data[product]
        test_product = test_data[product]
        predictions = list()
        history = [x for x in train_product[num_train_needed*-1:]]
        for t in range(len(test_product)):
            yhat = Croston_TSB(history, extra_periods=1, alpha=best_alpha, beta = best_beta)['Forecast'].iloc[-1]
            predictions.append(yhat)
            history.append(test_product[t])
        df_result_temp = pd.DataFrame({'Product': [product for count in range(len(test_product))],
                                        'Actual Data': test_product,
                                        'Forecast': predictions})
        df_result_best_params = df_result_best_params.append(df_result_temp, ignore_index=True)
            
    return df_result_best_params

In [ ]:
start = timer()

pattern_df_result_best_params = CrostonTSB_test_data(best_alpha, best_beta, sales_smooth, start_train_date, end_train_date, n_pred_days, num_train_needed, validation_training_ratio)

end = timer()

print('This line of code took {} minutes'.format((end-start) / 60))

This line of code took 0.13483726148333516 minutes


In [ ]:
df_result_final = pattern_df_result_best_params.groupby('Product').apply(get_metrics_result_all_params).reset_index()

In [ ]:
df_result_final

,Product,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
0,FOODS_1_004_CA_1_validation,0.710398,62.3678,71.2588,80.8317,4.098453,4.800791
1,FOODS_1_004_CA_3_validation,inf,inf,200.0000,NaN,0.930238,1.007915
2,FOODS_1_004_CA_4_validation,0.867910,52.2317,55.6739,73.9626,1.268483,1.644335
3,FOODS_1_012_CA_1_validation,0.689915,48.0755,55.2426,73.6431,1.751323,2.162670
4,FOODS_1_012_CA_2_validation,0.684552,40.2555,44.0081,36.6142,4.370599,5.118700
...,...,...,...,...,...,...,...
637,FOODS_3_818_CA_4_validation,0.544830,40.4648,45.6873,38.5471,0.838200,1.145495
638,FOODS_3_819_CA_3_validation,0.658640,52.8502,52.9749,113.6228,1.925257,2.289063
639,FOODS_3_820_CA_1_validation,0.576305,54.5408,66.3371,47.6559,1.285604,1.507597
640,FOODS_3_820_CA_3_validation,0.697637,71.9998,86.3709,41.3186,1.234282,1.487294


In [ ]:
df_result_final.to_csv('CrostonTSB_Smooth_Test_Data.csv')